In [3]:
from Libs.Processing import session_data

import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize

import datetime

import seaborn as sns
color = sns.color_palette()
from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

from collections import Counter

from sklearn.model_selection import GroupKFold
from sklearn.model_selection import train_test_split,TimeSeriesSplit
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder,LabelBinarizer,LabelEncoder,PolynomialFeatures,MinMaxScaler,minmax_scale

from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostRegressor, Pool

from  itertools import combinations

from sklearn.linear_model import LinearRegression, Ridge, LassoCV
from sklearn.metrics import mean_absolute_error, mean_squared_error,r2_score

import lightgbm as lgb

from sklearn.svm import SVR

%matplotlib inline
import matplotlib.pyplot as plt

from IPython.display import display
pd.options.display.max_columns = None

import gc

In [21]:
def plotCategoryRateBar(df, target, col_name, topN = np.nan):

    df_plot = df.groupby(col_name)[target].agg(['size','count'])
    df_plot['size'] = df_plot['size'] / df.shape[0]
    df_plot['count'] = df_plot['count'] / df[df[target].isnull() == False].shape[0]    
    if topN is not np.nan:
        df_plot = df_plot.sort_values("count", ascending=False).iloc[:topN , :]

    df_plot.sort_values("count").plot.barh()
        

In [4]:
df_train = pd.read_csv('2_flatten_train_v2.csv', sep=',',encoding ='utf8',dtype={'fullVisitorId': 'str'})
df_test = pd.read_csv('2_flatten_test_v2.csv', sep=',',encoding ='utf8',dtype={'fullVisitorId': 'str'})

In [3]:
session_data(df_train, df_test)

In [9]:
df_test['date'].max()

20181015

In [10]:
df_test['date'].min()

20180501

In [5]:
df_train['totals.transactionRevenue'].fillna(0, inplace=True)
df_train['totals.transactionRevenue'] = np.log1p(df_train['totals.transactionRevenue'].astype(float))
df_train['part'] = 'train'

In [6]:
df_test['totals.transactionRevenue'].fillna(0, inplace=True)
df_test['totals.transactionRevenue'] = np.log1p(df_test['totals.transactionRevenue'].astype(float))
df_test['part'] = 'test'

In [7]:
all_data = df_train.append(df_test, sort=False).reset_index(drop=True)

In [8]:
print(df_train.shape)
print(df_test.shape)

(1708337, 59)
(401589, 59)


In [9]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2109926 entries, 0 to 2109925
Data columns (total 59 columns):
channelGrouping                                      object
customDimensions                                     object
date                                                 int64
fullVisitorId                                        object
socialEngagementType                                 object
visitId                                              int64
visitNumber                                          int64
visitStartTime                                       int64
device.browser                                       object
device.browserSize                                   object
device.browserVersion                                object
device.deviceCategory                                object
device.flashVersion                                  object
device.isMobile                                      bool
device.language                                      

In [10]:
const_cols = [col for col in df_train.columns if df_train[col].nunique(dropna=False)==1 and col != 'part']
all_data.drop(const_cols, axis=1, inplace=True)

In [11]:
leakage_cols = ['totals.transactions', 'totals.totalTransactionRevenue']
all_data.drop(leakage_cols, axis=1, inplace=True)

In [12]:
null_count = all_data.isnull().sum().sort_values()
null_count = null_count[null_count>0]
type_cols = all_data[null_count.index].dtypes
should_fill = pd.concat([null_count, type_cols], axis=1)
should_fill

,0,1
totals.pageviews,340,float64
totals.newVisits,516431,float64
totals.sessionQualityDim,835274,float64
totals.bounces,1055670,float64
totals.timeOnSite,1057980,float64
trafficSource.keyword,1093006,object
trafficSource.referralPath,1142073,object
trafficSource.isTrueDirect,1426999,object
trafficSource.adContent,1643600,object
trafficSource.adwordsClickInfo.gclId,2023891,object


In [13]:
#fillna object feature
for col in type_cols[type_cols=='object'].index:
    all_data[col].fillna('unknown', inplace=True)

#fillna numeric features 
all_data['trafficSource.adwordsClickInfo.page'].fillna(1 ,inplace=True)
all_data['totals.pageviews'].fillna(1 ,inplace=True)
all_data['totals.newVisits'].fillna(0 , inplace=True)
all_data['totals.sessionQualityDim'].fillna(1, inplace=True)
all_data['totals.bounces'].fillna(0, inplace=True)
all_data['totals.timeOnSite'].fillna(all_data['totals.timeOnSite'].median(), inplace=True)

In [11]:
#all_data['trafficSource.adwordsClickInfo.gclId'].value_counts()

## Date


In [14]:
all_data['Start_date_time']=pd.to_datetime(all_data['visitStartTime'],unit='s')

all_data["_month"]   = all_data['Start_date_time'].dt.month
all_data["_day"]     = all_data['Start_date_time'].dt.day
all_data["_weekday"] = all_data['Start_date_time'].dt.weekday
all_data['_hour'] = all_data['Start_date_time'].dt.hour
all_data['_year'] = all_data['Start_date_time'].dt.hour

In [15]:
all_data['isMobile'] = True
all_data.loc[all_data['device.deviceCategory'] == 'desktop', 'isMobile'] = False

all_data['_adContentGMC'] = (all_data['trafficSource.adContent'] == 'Google Merchandise Collection').astype('uint8')

all_data['_withCampaign'] = (all_data['trafficSource.campaign'] != '(not set)').astype('uint8')

all_data['_refPath'] = (all_data['trafficSource.referralPath'] == '/').astype('uint8')

all_data.loc[~all_data['trafficSource.source'].isin(['(direct)','google']),'trafficSource.source'] = 'other'

In [37]:
# all_data.sort_values(['fullVisitorId', 'Start_date_time'], ascending=True, inplace=True)
# all_data['next_session_1'] = (all_data['Start_date_time'] - all_data[['fullVisitorId', 'Start_date_time']].groupby('fullVisitorId')['Start_date_time'].shift(1)
#     ).fillna(0).astype(np.int64) // 1e9 // 60 // 60
# all_data['next_session_2'] = (all_data['Start_date_time'] - all_data[['fullVisitorId', 'Start_date_time']].groupby('fullVisitorId')['Start_date_time'].shift(-1)
#     ).fillna(0).astype(np.int64) // 1e9 // 60 // 60
    
all_data['ratio_pageviews'] = all_data['totals.pageviews'] / all_data.groupby(['_day'])['totals.pageviews'].transform('sum')

In [16]:
all_data['real_date'] = pd.to_datetime(all_data['date'], format="%Y%m%d")

In [17]:
def periods(df):
    min_d = df['real_date'].min()
    max_d = df['real_date'].max()
    curr_d = min_d
    while curr_d  + pd.DateOffset(months=1) < max_d:
        new_d = curr_d + pd.DateOffset(months=2)
        last_d = curr_d
        if new_d > max_d: 
            last_d = max_d - pd.DateOffset(months=2)
            yield (last_d , max_d)
        else:
            yield (last_d , new_d)
        curr_d = curr_d + pd.DateOffset(months=1)

In [43]:
df_parts = []
for frame in periods(all_data):
    df_parts.append(all_data[(all_data['real_date'] >= frame[0]) & (all_data['real_date'] <= frame[1])])

In [19]:
df_parts[3].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 194460 entries, 31608 to 1664450
Data columns (total 49 columns):
channelGrouping                                 194460 non-null object
customDimensions                                194460 non-null object
date                                            194460 non-null int64
fullVisitorId                                   194460 non-null object
visitId                                         194460 non-null int64
visitNumber                                     194460 non-null int64
visitStartTime                                  194460 non-null int64
device.browser                                  194460 non-null object
device.deviceCategory                           194460 non-null object
device.isMobile                                 194460 non-null bool
device.operatingSystem                          194460 non-null object
geoNetwork.city                                 194460 non-null object
geoNetwork.continent                  

In [33]:
redundant_cols = ['Start_date_time','trafficSource.campaign', 'device.deviceCategory', 
                 'device.isMobile','trafficSource.referralPath','totals.bounces',
                  'totals.newVisits','visitStartTime',
                 'totals.timeOnSite','totals.pageviews','totals.hits','totals.sessionQualityDim', 'real_date']

In [34]:
drop_cols = ['date','fullVisitorId', 'totals.transactionRevenue', 'visitId', 'part']
cat_cols = [col for col in all_data.columns 
           if col not in (drop_cols + redundant_cols) and (all_data[col].dtype=='object' or all_data[col].dtype=='bool')
           ]

In [38]:
usr_cat_cols = cat_cols + ['_adContentGMC', '_withCampaign', '_refPath']
usr_num_cols = set(all_data.columns).difference(set(usr_cat_cols + drop_cols + redundant_cols + ['_day','_month','_hour','_weekday','_year']))


In [39]:
usr_num_cols

{'ratio_pageviews', 'trafficSource.adwordsClickInfo.page', 'visitNumber'}

In [ ]:
# trn_usr_cat = df_train[usr_cat_cols + ['fullVisitorId']].groupby(['fullVisitorId']).median()
# trn_usr_num = df_train[list(usr_num_cols) + ['fullVisitorId']].groupby(['fullVisitorId']).mean()
# trn_full_usr = pd.concat([trn_usr_cat,trn_usr_num], axis=1)

In [51]:
def create_user_dataset(df):
    
    df.sort_values(['fullVisitorId', 'Start_date_time'], ascending=True, inplace=True)
    df['next_session_1'] = (df['Start_date_time'] - df[['fullVisitorId', 'Start_date_time']].groupby('fullVisitorId')['Start_date_time'].shift(1)
        ).fillna(0).astype(np.int64) // 1e9 // 60 // 60
    df['next_session_2'] = (df['Start_date_time'] - df[['fullVisitorId', 'Start_date_time']].groupby('fullVisitorId')['Start_date_time'].shift(-1)
        ).fillna(0).astype(np.int64) // 1e9 // 60 // 60
    
    df_cat = df[usr_cat_cols + ['fullVisitorId']].groupby(['fullVisitorId']).median()
    df_num = df[list(usr_num_cols) + ['fullVisitorId', 'next_session_1' ,'next_session_2']].groupby(['fullVisitorId']).mean()
    df_full = pd.concat([df_cat,df_num], axis=1)
    
    df_full['target'] = df[['fullVisitorId', 'totals.transactionRevenue']].groupby('fullVisitorId').sum()
    
    for feature in ['totals.timeOnSite','totals.pageviews','totals.hits','totals.sessionQualityDim']:
    
        df_full['_userMean_' + feature] = df.groupby(['fullVisitorId'])[feature].mean()
    # max    
    for feature in ['totals.timeOnSite','visitNumber']:
    
        df_full['_userMax_' + feature] = df.groupby(['fullVisitorId'])[feature].max()
        
    return df_full
      

In [52]:
create_user_dataset(df_parts[3].copy())

,isMobile,_adContentGMC,_withCampaign,_refPath,visitNumber,trafficSource.adwordsClickInfo.page,ratio_pageviews,next_session_1,next_session_2,target,_userMean_totals.timeOnSite,_userMean_totals.pageviews,_userMean_totals.hits,_userMean_totals.sessionQualityDim,_userMax_totals.timeOnSite,_userMax_visitNumber
fullVisitorId,,,,,,,,,,,,,,,,
0000020424342248747,False,0.0,0.0,0.0,1.0,1.0,0.000051,0.0,0.000000,0.000000,297.000000,13.000000,17.0,1.0,297.0,1
0000174067426171406,True,0.0,0.0,0.0,1.5,1.0,0.000056,11.0,-11.500000,0.000000,559.500000,15.000000,17.5,1.0,1086.0,2
000033471059618621,False,0.0,0.0,0.0,1.0,1.0,0.000004,0.0,0.000000,0.000000,82.000000,1.000000,1.0,1.0,82.0,1
0000388388833742701,False,0.0,0.0,0.0,1.0,1.0,0.000007,0.0,0.000000,0.000000,6.000000,2.000000,2.0,1.0,6.0,1
0000426241577666985,False,0.0,0.0,0.0,1.0,1.0,0.000008,0.0,0.000000,0.000000,50.000000,2.000000,2.0,1.0,50.0,1
0000485418944539313,False,0.0,0.0,0.0,1.0,1.0,0.000004,0.0,0.000000,0.000000,82.000000,1.000000,1.0,1.0,82.0,1
0000500012543476818,False,0.0,0.0,1.0,1.0,1.0,0.000016,0.0,0.000000,0.000000,34.000000,4.000000,4.0,1.0,34.0,1
0000538867824729259,False,0.0,0.0,0.0,1.0,1.0,0.000004,0.0,0.000000,0.000000,82.000000,1.000000,1.0,1.0,82.0,1
0000649777500417577,True,0.0,0.0,0.0,1.5,1.0,0.000013,24.0,-24.500000,0.000000,172.500000,3.500000,3.5,1.0,263.0,2


In [22]:
# day, month features
# features for totals.hits
all_data['_meanHitsPD'] = all_data.groupby(['_day'])['totals.hits'].transform('mean')
all_data['_meanHitsPM'] = all_data.groupby(['_month'])['totals.hits'].transform('mean')
all_data['_sumHitsPD'] = all_data.groupby(['_day'])['totals.hits'].transform('sum')
all_data['_sumHitsPM'] = all_data.groupby(['_month'])['totals.hits'].transform('sum')
# features for totals.pageviews
all_data['_meanPagePD'] = all_data.groupby(['_day'])['totals.pageviews'].transform('mean')
all_data['_meanPagePM'] = all_data.groupby(['_month'])['totals.pageviews'].transform('mean')
all_data['_sumPagePD'] = all_data.groupby(['_day'])['totals.pageviews'].transform('sum')
all_data['_sumPagePM'] = all_data.groupby(['_month'])['totals.pageviews'].transform('sum')
# features for totals.timeOnSite
all_data['_meanTosPD'] = all_data.groupby(['_day'])['totals.timeOnSite'].transform('mean')
all_data['_meanTosPM'] = all_data.groupby(['_month'])['totals.timeOnSite'].transform('mean')
all_data['_sumTosPD'] = all_data.groupby(['_day'])['totals.timeOnSite'].transform('sum')
all_data['_sumTosPM'] = all_data.groupby(['_month'])['totals.timeOnSite'].transform('sum')
# features for totals.newVisits
all_data['_countNvisPD'] = all_data.groupby(['_day'])['totals.newVisits'].transform('count')
all_data['_countNvisPM'] = all_data.groupby(['_month'])['totals.newVisits'].transform('count')
all_data['_sumNvisPD'] = all_data.groupby(['_day'])['totals.newVisits'].transform('sum')
all_data['_sumNvisPM'] = all_data.groupby(['_month'])['totals.newVisits'].transform('sum')


# user features
# mean
for feature in ['totals.timeOnSite','totals.pageviews','totals.hits','totals.sessionQualityDim']:
    
    all_data['_userMean_' + feature] = all_data.groupby(['fullVisitorId'])[feature].transform('mean')
# max    
for feature in ['totals.timeOnSite','visitNumber']:
    
    all_data['_userMax_' + feature] = all_data.groupby(['fullVisitorId'])[feature].transform('max')
    

In [80]:
all_data.info(null_counts=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2109926 entries, 0 to 2109925
Data columns (total 70 columns):
channelGrouping                                 2109926 non-null object
customDimensions                                2109926 non-null object
date                                            2109926 non-null int64
fullVisitorId                                   2109926 non-null object
visitId                                         2109926 non-null int64
visitNumber                                     2109926 non-null int64
visitStartTime                                  2109926 non-null int64
device.browser                                  2109926 non-null object
device.deviceCategory                           2109926 non-null object
device.isMobile                                 2109926 non-null bool
device.operatingSystem                          2109926 non-null object
geoNetwork.city                                 2109926 non-null object
geoNetwork.continent         

In [81]:
redundant_cols = ['Start_date_time','trafficSource.campaign', 'device.deviceCategory', 
                 'device.isMobile','trafficSource.referralPath','totals.bounces',
                  'totals.newVisits','visitStartTime',
                 'totals.timeOnSite','totals.pageviews','totals.hits','totals.sessionQualityDim']
all_data.drop(redundant_cols, axis=1, inplace=True)

In [82]:
all_data.to_pickle('Prepared_session_data')